In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import regex as re
import requests, time

In [ ]:
# Primeiramente vamos criar uma lista com todos os filmes lançados entre 01/01/2000 e 31/12/2021 (total de 247,584 filmes)
url_base = 'https://www.imdb.com/search/title/?title_type=feature,tv_movie&release_date=2020-01-01,2021-12-31'
response = requests.get(url_base)
response

In [ ]:
# salvando o HTML para poder abrir em um editor e ficar mais fácil de trabalhar
soup = BeautifulSoup(response.content, features = 'lxml')
with open("url_base.html", "w", encoding='utf-8') as file:
    file.write(str(soup))

Coletar as informações da página antes de criar um loop
Dessa página precisamos tirar:
1. Movie Link
2. Next page link

In [ ]:
list_detail = soup.find_all('div', attrs = {'class':'lister-item-content'})
link = list_detail[0]
linkID = link.a['href']
linkMovie = f'https://www.imdb.com{linkID}'
linkMovie

In [ ]:
linkNext1 = soup.find_all('a', attrs = {'class':'lister-page-next next-page'})[0]
linknext = linkNext1['href']
linkNext = f'https://www.imdb.com{linknext}'
linkNext

In [ ]:
# Criando uma lista que contenha todos os links de filmes na página     
list_detail = soup.find_all('div', attrs = {'class':'lister-item-content'})
url_list = ["https://www.imdb.com" + link.a['href'] for link in list_detail]
print(url_list)

In [ ]:
url = 'https://www.imdb.com/search/title/?title_type=feature,tv_movie&release_date=2020-01-01,2021-12-31'
url_list = []

try:
    for i in range (0,541):
        if i <= 539:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, features = 'lxml')

            list_detail = soup.find_all('div', attrs = {'class':'lister-item-content'})
            url_list_now = ["https://www.imdb.com" + link.a['href'] for link in list_detail]
            url_list.append(url_list_now)
            
            try:
                linkNext1 = soup.find_all('a', attrs = {'class':'lister-page-next next-page'})[0]
                linknext = linkNext1['href']
                url = f'https://www.imdb.com{linknext}'
            except:
                url = url

        else:   
            flat_list = [item for lista in url_list for item in lista]
except:
    print(f'issue with this link: {url}')

In [ ]:
# Checando se há links duplicados
print(len(flat_list))
set_links = list(set(flat_list))
print(len(set_links))

In [ ]:
# salvar em um arquivo, pois o código era muito pesado e não quero ter que rodar toda vez
movies_list = pd.DataFrame (set_links, columns = ['imdb_link'])
movies_list.to_csv('lista_imdb_links_2020_2021.csv', index=False)

In [ ]:
# Pegar dados da página do filme
link = 'https://www.imdb.com/title/tt9032400/?ref_=adv_li_tt'
response = requests.get(link)
soup = BeautifulSoup(response.content, features = 'lxml')

# 1. Movie ID
imdbID = link.replace('https://www.imdb.com/title/','').replace('/?ref_=adv_li_tt','')
print(imdbID)
# 2. link para a página de avaliações
rating_link = f'{link}ratings/?ref_=tt_ov_rt'
print(rating_link)
# 3. nome do filme
movie_name = soup.title.text
movie_name = movie_name.replace(' - IMDb','')
print(movie_name)
# 4. ano de lançamento do filme
try:
    year = soup.find_all('ul', attrs = {'data-testid':"hero-title-block__metadata"})[0].text
    year = re.findall(r'[0-9]{4}', year)
    year = year[0]
except:
    year = ''
print(year)
# 5. nota do IMDB
try:
    imdb_rating = soup.find_all('span', attrs = {'class':"AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV"})[0].text
except:
    imdb_rating = ''
print(imdb_rating)
# 6. quantidade de votos
try:
    user_votes = soup.find_all('div', attrs = {'class':"AggregateRatingButton__TotalRatingAmount-sc-1ll29m0-3 jkCVKJ"})[0].text
except:
    user_votes = ''
print(user_votes)
# 7. nome do diretor
try:
    director = soup.find_all('a', attrs = {'class':"ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"})[0].text
except:
    director = ''
print(director)
# 8. prêmios ganhos e nomeações (testei em caso o filme não tenha premiações)
try:
    awards = soup.find_all('li', attrs = {'data-testid':"award_information"})[0].text
except:
    awards = ''
print(awards)    
# 9. oscars
oscars = re.findall(r'(\d+)(?=\s*Oscar)', awards)[0]
print(oscars)
# 10. prêmios ganhos
awards_won = re.findall(r'(\d+)(?=\s*win)', awards)[0]
print(awards_won)
# 11. nomeações
awards_nominated = re.findall(r'(\d+)(?=\s*nomination)', awards)[0]
print(awards_nominated)
# 12. gêneros
try:
    genres_html = soup.find_all('div', attrs = {'data-testid':"genres"})[0]
    genre = genres_html.find_all('a', attrs = {'class':'GenresAndPlot__GenreChip-sc-cum89p-3 LKJMs ipc-chip ipc-chip--on-baseAlt'})
    genres = [x.text for x in genre]
except:
    genres = ''
print(genres)
# 13. país
try:
    country_htlm = soup.find_all('li', attrs = {'data-testid':"title-details-origin"})[0]
    countries = country_htlm.find_all('a', attrs = {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
    country = [x.text for x in countries]
except:
    country = ''
print(country)
# 14. língua
try:
    languages_htlm = soup.find_all('li', attrs = {'data-testid':"title-details-languages"})[0]
    language = languages_htlm.find_all('a', attrs = {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
    languages = [x.text for x in language]
except:
    languages = ''
print(languages)
# 15. orçamento
try:
    boxoffice_budget = soup.find_all('li', attrs = {'data-testid':"title-boxoffice-budget"})[0].text
    boxoffice_budget = boxoffice_budget.replace('Budget','').replace(' (estimated)','')
    boxoffice_budget = re.findall(r'[0-9]*', boxoffice_budget)
    boxoffice_budget = ''.join(str(x) for x in boxoffice_budget)
except:
    boxoffice_budget = ''
print(boxoffice_budget)
# 16. receita
try:
    gross_worldwide = soup.find_all('li', attrs = {'data-testid':"title-boxoffice-cumulativeworldwidegross"})[0].text
    gross_worldwide = gross_worldwide.replace('Gross worldwide','').replace(' (estimated)','')
    gross_worldwide = re.findall(r'[0-9]*', gross_worldwide)
    gross_worldwide = ''.join(str(x) for x in gross_worldwide)
except:
    gross_worldwide = ''
print(gross_worldwide)

In [ ]:
flat_list_df = pd.read_csv ('lista_imdb_links_2020_2021.csv')
flat_list_df.head(2)
flat_list = flat_list_df['imdb_link'].tolist()

In [ ]:
data = []

for link in flat_list:
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, features = 'lxml')
        
        # get the data from the movie IMDB page
        imdbID = link.replace('https://www.imdb.com/title/','').replace('/','')
        # 2. link para a página de avaliações
        rating_link = f'{link}ratings/?ref_=tt_ov_rt'
        # 3. nome do filme
        movie_name = soup.title.text
        movie_name = movie_name.replace(' - IMDb','')
        # 4. ano de lançamento do filme
        try:
            year = soup.find_all('ul', attrs = {'data-testid':"hero-title-block__metadata"})[0].text
            year = re.findall(r'[0-9]{4}', year)
            year = year[0]
        except:
            year = ''
        # 5. nota do IMDB
        try:
            imdb_rating = soup.find_all('span', attrs = {'class':"AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV"})[0].text
        except:
            imdb_rating = ''
        # 6. quantidade de votos
        try:
            user_votes = soup.find_all('div', attrs = {'class':"AggregateRatingButton__TotalRatingAmount-sc-1ll29m0-3 jkCVKJ"})[0].text
        except:
            user_votes = ''
        # 7. nome do diretor
        try:
            director = soup.find_all('a', attrs = {'class':"ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"})[0].text
        except:
            director = ''
        # 8. prêmios ganhos e nomeações (testei em caso o filme não tenha premiações)
        try:
            awards = soup.find_all('li', attrs = {'data-testid':"award_information"})[0].text
        except:
            awards = ''
        # 9. oscars
        try:
            oscars = re.findall(r'(\d+)(?=\s*Oscar)', awards)[0]
        except:
            oscars = 0
        # 10. prêmios ganhos
        try:
            awards_won = re.findall(r'(\d+)(?=\s*win)', awards)[0]
        except:
            awards_won = 0
        # 11. nomeações
        try:
            awards_nominated = re.findall(r'(\d+)(?=\s*nomination)', awards)[0]
        except:
            awards_nominated = 0
        # 12. gêneros
        try:
            genres_html = soup.find_all('div', attrs = {'data-testid':"genres"})[0]
            genre = genres_html.find_all('a', attrs = {'class':'GenresAndPlot__GenreChip-sc-cum89p-3 LKJMs ipc-chip ipc-chip--on-baseAlt'})
            genres = [x.text for x in genre]
        except:
            genres = ''
        # 13. país
        try:
            country_htlm = soup.find_all('li', attrs = {'data-testid':"title-details-origin"})[0]
            countries = country_htlm.find_all('a', attrs = {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
            country = [x.text for x in countries]
        except:
            country = ''
        # 14. língua
        try:
            languages_htlm = soup.find_all('li', attrs = {'data-testid':"title-details-languages"})[0]
            language = languages_htlm.find_all('a', attrs = {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
            languages = [x.text for x in language]
        except:
            languages = ''
        # 15. orçamento
        try:
            boxoffice_budget = soup.find_all('li', attrs = {'data-testid':"title-boxoffice-budget"})[0].text
            boxoffice_budget = boxoffice_budget.replace('Budget','').replace(' (estimated)','')
            boxoffice_budget = re.findall(r'[0-9]*', boxoffice_budget)
            boxoffice_budget = ''.join(str(x) for x in boxoffice_budget)
        except:
            boxoffice_budget = ''
        # 16. receita
        try:
            gross_worldwide = soup.find_all('li', attrs = {'data-testid':"title-boxoffice-cumulativeworldwidegross"})[0].text
            gross_worldwide = gross_worldwide.replace('Gross worldwide','').replace(' (estimated)','')
            gross_worldwide = re.findall(r'[0-9]*', gross_worldwide)
            gross_worldwide = ''.join(str(x) for x in gross_worldwide)
        except:
            gross_worldwide = ''
        
        # Create a dictionary
        data.append({'imdbID': imdbID,
                     'movie_name':movie_name,
                     'year':year,
                     'imdb_rating':imdb_rating,
                     'rating_link':rating_link,
                     'user_votes':user_votes,
                     'director':director,
                     'oscars':oscars,
                     'awards_won':awards_won,
                     'awards_nominated':awards_nominated,
                     'genres':genres,
                     'country':country,
                     'languages':languages,
                     'boxoffice_budget':boxoffice_budget,
                     'gross_worldwide':gross_worldwide})
        
    except:
        print(f'issue with this link: {link}')

In [ ]:
# salvar em um arquivo, pois o código era muito pesado e não quero ter que rodar toda vez
movies_data = pd.DataFrame(data)
movies_data.to_csv('movies_data_imdb.csv', index=False)

In [ ]:
# Creating soup
link_erro = []

for link in flat_list:
    
    try:
        rating_link = f'{link}ratings/'
        response = requests.get(rating_link)
        soup = BeautifulSoup(response.content, features = 'lxml')
        
        # Extracting Movie name
        imdbID = link.replace('https://www.imdb.com/title/','').replace('/','')
        
        # Extracting table
        tables = soup.find_all('table')[1]
        
        # Extracting columns' names
        colname = tables.find_all('div', attrs = {'class':'tableHeadings'})
        colnames = [col.text.split("\n") for col in colname]
        flat_colnames = [item for sublist in colnames for item in sublist]
        
        # Extracting all rows
        row = tables.find_all('div', attrs = {'class':'bigcell'})
        rows = [r.text.split("\n") for r in row]
        flat_rows = [item for sublist in rows for item in sublist]
        
        splited = []
        len_l = len(flat_rows)
        for i in range(3):
            start = int(i*len_l/3)
            end = int((i+1)*len_l/3)
            splited.append(flat_rows[start:end])
        
        # Extracting columns gender
        col_gender = tables.find_all('div', attrs = {'class':'leftAligned'})
        col_gender = [i.text for i in col_gender]
        
        df = pd.DataFrame(splited, columns = flat_colnames)
        df.insert(0, 'Movie Id', imdbID)
        df.insert(1, 'Gender', col_gender)
        
        try:
            imdb_ratings = pd.concat([imdb_ratings,df])
        except:
            imdb_ratings = df
            
    except:
        link_erro.append(rating_link)

In [ ]:
# salvar em um arquivo, pois o código era muito pesado e não quero ter que rodar toda vez
imdb_ratings.to_csv('movies_ratings_imdb.csv', index=False)